In [1]:
#this is the same code we used for a CNN predicting a number based on a handmade drawing of it
#for this, i forked another repository: https://github.com/JosePabloGFnl/exportacion-numeros

import tensorflow as tf
import tensorflow_datasets as tfds

#MNIST data set
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

nombres_clases = metadatos.features['label'].names

#normalizing function, makes it faster
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #0-255 to 0-1
  return imagenes, etiquetas

datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

#creating the model
modelo = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax) 
])

#Compiling the model
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

num_ej_entrenamiento = metadatos.splits["train"].num_examples
num_ej_pruebas = metadatos.splits["test"].num_examples

TAMANO_LOTE = 32

#shuffling
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

import math

#training
historial = modelo.fit(datos_entrenamiento, epochs=5, steps_per_epoch= math.ceil(num_ej_entrenamiento/TAMANO_LOTE))

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Epoch 1/5
1875/1875 [==============================] - 51s 24ms/step - loss: 0.2937 - accuracy: 0.9082
Epoch 2/5
1875/1875 [==============================] - 41s 22ms/step - loss: 0.1131 - accuracy: 0.9663
Epoch 3/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.0845 - accuracy: 0.9753
Epoch 4/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.0752 - accuracy: 0.9778
Epoch 5/5
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0638 - accuracy: 0.9807


In [2]:
#exporting the model to h5
modelo.save('numeros.h5')

In [3]:
!ls

'~'   numeros.h5   sample_data


In [4]:
!pip install tensorflow js

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114 kB 53.6 MB/s 
  Created wheel for js: filename=js-1.0-py3-none-any.whl size=2907 sha256=6ca839466808093b2d45c35e2cd0f05bf0b22bb25e5faf11a0864cb9caa3c6c9
  Stored in directory: /root/.cache/pip/wheels/bf/92/56/6597c8f32980ee64af7e864796e8d7423eb67eafff5415a4ae
  Created wheel for shutilwhich: filename=shutilwhich-1.1.0-py3-none-any.whl size=2781 sha256=a9ab1b63d62cd78b9779aa46cd4bdf96a699112215292231be24103672cb3c40
  Stored in directory: /root/.cache/pip/wheels/4c/c0/ce/5a47f35186acbe73e38811df30daaf798c81a7599bb2d5995a
Successfully built js shutilwhich


In [5]:
#folder with output files
!mkdir carpeta_salida

In [6]:
#export to output folder
!tensorflowjs_converter --input_format keras numeros.h5 carpeta_salida

/bin/bash: tensorflowjs_converter: command not found


In [7]:
!ls carpeta_salida

In [8]:
#note, if tensorflow does not work in the page, do the following:
#open cmd > cd [folder] > python -m http.server 8000
#in the address, localhost:8000/index.html